# 4.2. 머신러닝 모델 평가
+ 훈련에 사용한 데이터로 모델을 평가하지 않는 이유는 **과대적합** 때문. 즉 훈련 데이터의 성능에 비해 처음 본 데이터에 대한 성능이 좋아지지 않거나 나빠지는 것이 문제. 훈련 데이터의 성능은 훈련이 진행될수록 항상 증가됨.
+ 머신러닝의 목표는 처음 본 데이터에서 잘 작동하는 **일반화된** 모델을 얻는 것임. 이런 측면에서 과대적합은 주요 장애물임.
+ 이 절에서는 일반화, 즉 머신 러닝 모델의 성능을 어떻게 측정하는지를 다룰 것임.

## <span style="color:orange"> 4.2.1. </span> <span style="color:black">훈련, 검증, 테스트 세트</span>
+ 모델 평가의 핵심은 가용한 데이터를 항상 훈련, 검증, 테스트 3개의 세트로 나누는 것임.
+ 훈련 세트에서 모델을 훈련하고 검증 세트에서 모델을 평가함. 모델을 출시할 준비가 되면 테스트 세트에서 최종적으로 딱 한번 모델을 테스트함.
+ 훈련 세트와 테스트 세트 두 개만 사용하지 않는 이유는 모델의 설정을 튜닝(tuning)하기 때문임.
    + 예를 들어, 층의 수나 유닛의 수를 조절할 수 있음: **하이퍼파마리미터(hyperparameter)** 조절.
    + 검증 세트에서 모델의 성능을 평가하여 이런 튜닝-좋은 파라미터 값을 찾는 것-을 수행함.
+ 검증 세트의 성능을 기반으로 모델의 설정을 튜닝하면 검증 세트로 모델을 직접 훈련하지 않더라도 **검증 세트에 과대적합**될 수 있음.
    + 이런 현상은 **정보 누설**(information lack)에 기인함. 검증 세트의 모델 성능에 기반하여 모델의 하이퍼파라미터를 조정할 때마다 검증 데이터에 관한 정보가 모델로 새는 것임.
+ 정보 누설을 고려하여 완전히 새로운 데이터 세트를 이용하여 모델의 일반화 성능을 평가하는 것이 바람직함.
+ 데이터가 적을 때 훈련, 검증, 테스트 세트로 나누는 경우에 이용할 수 있는 몇 가지 방법이 있음: **단순 홀드아웃 검증**(hold-out validation), **K-겹 교차 검증**(K-fold cross-validation), **셔플링을 사용한 반복 K-겹 교차 검증**(iterated K-fold cross-validation).

### 단순 홀드 아웃 검증

+ 데이터의 일정량을 테스트 세트로 떼어 놓고 남은 데이터에서 훈련하고 테스트 세트로 평가하는 것.
+ 사이킷런의 train_test_split()함수를 이용하는 것이 편리함.
+ 이 평가 방법은 데이터가 적을 때 검증 세트와 테스트 세트의 샘플이 너무 적어 주어진 전체 데이터를 통계적으로 대표하지 못한다는 단점이 있음. 다시 말해, 데이터가 바뀔때마다 모델의 성능이 크게 변할 수 있다는 것임.

*단순 홀드아웃 검증 분할*

<a href="https://ibb.co/QNNnNGh"><img src="https://i.ibb.co/3NNWNHV/87083-D26-27-C7-462-E-8-CA5-B8-A510-E6149-B-png.jpg" alt="87083-D26-27-C7-462-E-8-CA5-B8-A510-E6149-B-png" width="300" height="300" border="0" align = 'left'></a>

In [ ]:
# 홀드아웃 검증 구현 예
# 테스트 세트는 이미 떼어 놓은 것을 가정함.

num_validation_samples = 10000

np.random.shuffle(data) # 데이터를 섞는 것(셔플링)이 일반적으로 좋음.

validation_data = data[:num_validation_samples] # 검증 세트 생성
data = data[num_validation_samples:] 

traning_data = data[:] # 훈련세트 생성

# 훈련세트에서 모델을 훈련하고 검증 세트로 평가
model = get_model()
model.train(traning_data)
validation_score = model.evaluate(validation_data)

# 여기서 모델을 튜닝하고,
# 다시 훈련하고, 평가하고, 또 다시 튜닝하고...

model = get_model()
model.train(np.condatenate([traning_data,
                           validation_data]))
test_score = model.evaluate(test_data)

### K-겹 교차 검증

+ 데이터를 동일한 크기를 가진 K개의 분할로 나눔. 각 분할 i에 대해 남은 K-1개의 분할로 모델을 훈련하고 분할 i에서 모델을 평가함.
+ 최종 점수는 이렇게 얻은 K개의 점수를 평균함.
+ 이 방법은 모델의 성능이 데이터의 분할에 따라 편차가 클 때 도움이 됨.
+ 홀드아웃 검증처럼 모델의 튜닝에 별개의 검증 세트를 사용하게 됨.

*3-겹 교차 검증*

<a href="https://imgbb.com/"><img src="https://i.ibb.co/cXtQTSs/87083-D26-27-C7-462-E-8-CA5-B8-A510-E6149-B-png.jpg" alt="87083-D26-27-C7-462-E-8-CA5-B8-A510-E6149-B-png" width="500" height="500" border="0" align = 'left'></a>

In [ ]:
# K-겹 교차 검증 구현 예

k = 4
num_validation_samples = len(daata) // k

np.random.shuffle(data)

validation_scores = []
for fold in range(k):
    # 검증 데이터 부분 선택
    validation_data = data[num_validation_samples * fold:
                          num_validation_samples * (fold+1)]
    
    # 남은 데이터를 훈련 데이터로 사용. 리스트에서 + 연산자는 두 연산자를 연결함.
    training_data = data[:num_validation_samples * fold] + data[num_validation_samples * (fold+1):]
    
    model = get_model() # 훈련되지 않은 새로운 모델 생성
    model.train(traning_daata)
    validation_score = model.evaluate(validation_data)
    validation_scores.append(validation_score)
    
    validation_score = np.average(validation_scores) # 검증점수: K개 폴드의 검증 점수 평균
    
    # 테스트 데이터를 제외한 전체 데이터로 최종 모델 훈련
    model = get_model()
    model.train(data)
    test_scores = model.evaluate(test_data)

### 셔플링을 사용한 반복 K-겹 교차 검증

+ 이 방법은 비교적 가용 데이터가 적고 가능한 정확하게 모델을 평가하고자 할 때 사용함.
+ K-겹 교차 검증을 여러번 적용하되 K개의 분할로 나누기 전에 매번 데이터를 무작위로 섞음. 최종 점수는 모든 K-겹 교차 검증을 실행해서 얻은 점수의 평균이 됨.
+ 결국 P×K(P는 반복 횟수)의 모델을 훈련하고 평가하므로 시간이 많이 듦. 사이킷런 0.19 버전에 추가된 RepeatedKFold(회귀)와 RepeatedStratifiedKfold(분류) 클래스를 cross_validate 함수에 적용하여 구현할 수 있음. 이와 관련된 예제는 다음 링크를 참고할 것: https://bit.ly/2rSVwjB

## <span style="color:orange"> 4.2.2. </span> <span style="color:black">기억해야 할 것</span>

평가 방식을 선택할 때 다음 사항을 유념해야 함
+ **대표성 있는 데이터** : 훈련 세트와 테스트 세트가 주어진 데이터에 대한 대표성이 있어야 함.일반적으로 훈련 세트와 테스트 세트로 나누기 전에 데이터를 무작위로 섞음. 사이킷런의 train_test_split() 함수는 stratify 매개변수로 타깃 레이블을 전달받아 계층별 분할을 수행할 수 있음.
+ **시간의 방향** : 과거로부터 미래를 예측하려 한다면 데이터를 분할하기 전에 무작위로 섞어서는 절대 안 됨. 이렇게 하면 미래의 정보가 누설되기 때문임. 이런 문제에서는 훈련 세트에 있는 데이터보다 테스트 세트에 있는 모든 데이터가 미래의 것어야 함.
+ **데이터 중복** : 데이터 중복은 꼭 피해야 함. 훈련 데이터의 일부로 테스트하는 경우가 초래할 수 있음. 사이킷런의 GroupKFold 클래스를 cross_validate() 함수에 적용해야 함.